# Forward Kinematics with Denavit-Hartenberg

In [1]:
import dh

# Solves the forward kinemetics in D-H conventions, returns the frames and transformations for each joints
def fk_dh(dh_params, frames_at_rest):
    # Get the number of links
    assert(len(dh_params) == len(frames_at_rest)-1)
    joints = range(len(dh_params))
    links = range(len(dh_params) + 1)

    # Compute the frames along the chain
    frames = [np.eye(4) for i in links]
    for i in joints:
        dh_matrix = dh.to_matrix(dh_params[i])
        frames[i+1] = frames[i] @ dh_matrix

    # Compute the transforms along the chain
    transforms = [frames[i] @ np.linalg.inv(frames_at_rest[i]) for i in links]

    return (frames, transforms)

# Forward Kinematics with Power-of-Exponential

In [2]:
import modern_robotics as mr

# Solves the forward kinemetics in PoE conventions, returns the frames and transformations for each joints
def fk_poe(screw_axes, screw_angles, frames_at_rest):
    # Get the number of links
    assert(len(screw_axes) == len(screw_angles) == len(frames_at_rest)-1)
    joints = range(len(screw_angles))
    links = range(len(screw_angles) + 1)

    # Compute the transformations along the chain
    transforms = [np.eye(4) for i in links]
    for i in joints:
        exp_screw_axis_angle = mr.MatrixExp6(mr.VecTose3(screw_axes[i] * screw_angles[i]))
        transforms[i+1] = transforms[i] @ exp_screw_axis_angle
    
    # Compute the frames along the chain
    frames = [transforms[i] @ frames_at_rest[i] for i in links]

    return (frames, transforms)

# Testing the forward kinematics
## Initializing the parameters

In [3]:
import numpy as np

# Unit of distance = 1 m
H1 = 400e-3
H3 = 35e-3
L1 = 25e-3
L2 = 455e-3
L3 = 420e-3
L4 = 80e-3

num_links = 7
num_joints = 6

# The joints will be refered to with an index from 0 to 5 instead of 1 to 6. Deal with it
dh_params = [
    dh.DHparam(-H1,    0,          L1, np.pi/2 ), # Frame 0 to 1
    dh.DHparam(0,      0,          L2, 0       ), # Frame 1 to 2
    dh.DHparam(0,      -np.pi/2,   H3, np.pi/2 ), # Frame 2 to 3
    dh.DHparam(-L3,    0,          0,  -np.pi/2), # Frame 3 to 4
    dh.DHparam(0,      0,          0,  np.pi/2 ), # Frame 4 to 5
    dh.DHparam(-L4,    0,          0,  0       ), # Frame 5 to 6
]

# Screw axes in frame 0 (global) for joints 1 to 6
screw_axes = [
    np.array([  0,  0,  1,  0,      0,      0,          ]), # S_1
    np.array([  0,  -1, 0,  -H1,    0,      -L1,        ]), # S_2
    np.array([  0,  -1, 0,  -H1,    0,      -L1-L2,     ]), # S_3
    np.array([  -1, 0,  0,  0,      H1+H3,  0,          ]), # S_4
    np.array([  0,  -1, 0,  -H1-H3, 0,      -L1-L2-L3   ]), # S_5
    np.array([  -1, 0,  0,  0,      H1+H3,  0,          ]), # S_6
]

# Frames 0 to 6 in the DH convention at resting position (when all joints are 0)
frames_at_rest = [
    np.eye(4), # M_0
    np.array([[1, 0, 0, L1],            [0, 0, -1, 0],  [0, 1, 0, -H1],     [0, 0, 0, 1]]), # M_1
    np.array([[1, 0, 0, L1+L2],         [0, 0, -1, 0],  [0, 1, 0, -H1],     [0, 0, 0, 1]]), # M_2
    np.array([[0, 0, -1, L1+L2],        [0, -1, 0, 0],  [-1, 0, 0, -H1-H3], [0, 0, 0, 1]]), # M_3
    np.array([[0, 1, 0, L1+L2+L3],      [0, 0, -1, 0],  [-1, 0, 0, -H1-H3], [0, 0, 0, 1]]), # M_4
    np.array([[0, 0, -1, L1+L2+L3],     [0, -1, 0, 0],  [-1, 0, 0, -H1-H3], [0, 0, 0, 1]]), # M_5
    np.array([[0, 0, -1, L1+L2+L3+L4],  [0, -1, 0, 0],  [-1, 0, 0, -H1-H3], [0, 0, 0, 1]]), # M_6 = M
]

## Visualizing

In [4]:
import robot_scene

# Create the scene objects
scene = robot_scene.create_scene()

/usr/local/anaconda3/envs/Robotikk/lib/python3.8/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [5]:
from IPython.display import display
import ipywidgets

# Update the links transforms
def show_robot(scene, transforms, must_show):
    for (link, transform) in zip(scene.links, transforms):
        link.matrix = transform.flatten(order="F").tolist()
        link.visible = must_show

# Update the frames transforms
def show_frames(scene, frames, must_show):
    for (trihedron, frame) in zip(scene.trihedron, frames):
        trihedron.matrix = frame.flatten(order="F").tolist()
        trihedron.visible = must_show

# Update the screw axes
def show_axes(scene, frames, must_show):
    for (axis, frame) in zip(scene.axes, frames):
        axis.matrix = frame.flatten(order="F").tolist()
        axis.visible = must_show
        
# Interaction loop
def interact_fk(mode, j1, j2, j3, j4, j5, j6, must_show_robot, must_show_frames, must_show_axes):
    joint_angles = np.radians([j1, j2, j3, j4, j5, j6])
    
    # Do the forward kinematics in D-H conventions
    if mode == "fk_dh" or mode == "compare":
        updated_dh = [dh.DHparam(x.d, x.theta+j, x.a, x.alpha) for (x, j) in zip(dh_params, joint_angles)]
        (frames, transforms) = (frames_dh, transforms_dh) = fk_dh(updated_dh, frames_at_rest)
    
    # Do the forward kinematics in PoE conventions
    if mode == "fk_poe" or mode == "compare":
        (frames, transforms) = (frames_poe, transforms_poe) = fk_poe(screw_axes, joint_angles, frames_at_rest)
    
    # Do both and compare
    if mode == "compare":
        print("PoE and DH similar:", np.allclose(frames_dh[-1], frames_poe[-1]))
        
    # Update the scene
    print("End effector:\n", frames[-1])
    show_robot(scene, transforms, must_show_robot)
    show_frames(scene, frames, must_show_frames)
    show_axes(scene, frames, must_show_axes)

# Start the interaction
interactive = ipywidgets.interactive(
    interact_fk,
    mode=ipywidgets.ToggleButtons(options=["fk_poe", "fk_dh", "compare"]),
    j1=ipywidgets.IntSlider(description="J1 (°)", min=-180, max=180, step=1, value=0),
    j2=ipywidgets.IntSlider(description="J2 (°)", min=-180, max=180, step=1, value=0),
    j3=ipywidgets.IntSlider(description="J3 (°)", min=-180, max=180, step=1, value=0),
    j4=ipywidgets.IntSlider(description="J4 (°)", min=-180, max=180, step=1, value=0),
    j5=ipywidgets.IntSlider(description="J5 (°)", min=-180, max=180, step=1, value=0),
    j6=ipywidgets.IntSlider(description="J6 (°)", min=-180, max=180, step=1, value=0),
    must_show_robot=ipywidgets.Checkbox(description="Show Robot", value=True),
    must_show_frames=ipywidgets.Checkbox(description="Show Frames", value=False),
    must_show_axes=ipywidgets.Checkbox(description="Show Axes", value=False),
)

display(ipywidgets.HBox([
    scene.viewer.renderer,
    ipywidgets.VBox(interactive.children)
]))


# Testing the inverse kinematics
## Inverse kinematics analytic solution

In [6]:
# ref: https://www.geometrictools.com/Documentation/EulerAngles.pdf
def negXYX_EulerAngles(R):
    r11, r12, r13, r21, r22, r23, r31 = R[0, 0], R[0, 1], R[0, 2],R[1, 0], R[1, 1], R[1, 2], R[2, 0]
    if (r11 < 1):
        if (r11 > -1):
            thetaX0 = -np.arctan2(r21, -r31) 
            thetaY = -np.arccos(r11) 
            thetaX1 = -np.arctan2(r12,r13)

        else: # r11 = −1 
            # Not a unique solution: thetaX1 − thetaX0 = atan2(−r12,r11) 
            thetaX0 = np.arctan2(-r23,r22);
            thetaY = -np.pi
            thetaX1 = 0
    
    else: #r11 = 1 
        #Not a unique solution: thetaX1 + thetaX0 = atan2(−r12,r11) 
        thetaX0 = -np.arctan2(-r23,r22)
        thetaY = 0
        thetaX1 = 0
        
    return thetaX0, thetaY, thetaX1

def arm_fk(theta1, theta2, theta3):
    c_theta1 = np.cos(theta1)
    s_theta1 = np.sin(theta1)
    R1 = np.array([[c_theta1, -s_theta1, 0], 
                   [s_theta1,  c_theta1, 0], 
                   [    0   ,     0    , 1]])
    
    c_theta2 = np.cos(theta2)
    s_theta2 = np.sin(theta2)
    R2 = np.array([[c_theta2,  0, s_theta2], 
                   [    0   ,  1,     0   ], 
                   [-s_theta2, 0, c_theta2]])
    
    c_theta3 = np.cos(theta3)
    s_theta3 = np.sin(theta3)
    R3 = np.array([[c_theta3,  0, s_theta3], 
                   [    0,     1,     0   ], 
                   [-s_theta3, 0, c_theta3]])
    
    return R1 @ R2 @ R3

def ik_analytic(end_effector_frame):
    # Extract the desired point P and pose R
    R = end_effector_frame[0:3, 0:3]
    P = end_effector_frame[0:3, 3]
    
    # W = wrist center
    Wx = P[0] + L4*R[0, 2] # Wrist center is translated L4 along the z-axis from the desired pose
    Wy = P[1] + L4*R[1, 2] 
    Wz = -P[2] - L4*R[2, 2]
        
    # Found theta 1
    theta1 = np.arctan2(Wy, Wx)
    
    R2 = np.hypot(Wx, Wy) - L1
    Z2 = Wz - H1
    C2 = np.hypot(R2, Z2)
    C3 = np.hypot(L3, H3)
    phi1 = np.arctan(H3 / L3)
    beta = np.arccos((L2**2 + C3**2 - C2**2) / (2*L2*C3))
    phi2 = np.pi - beta
    R3 = C3 * np.cos(phi2)
    Z3 = C3 * np.sin(phi2)
    gamma = np.arctan2(Z2, R2)
    alpha = np.arctan2(Z3, L2 + R3)
    M = np.array([[0, 0, -1], [0, -1, 0], [-1, 0, 0]])
    
    # Elbow up solution
    theta2_u = gamma + alpha
    theta3_u = -phi1 - phi2
    
    # Wrist rotation
    wrist_rotation = arm_fk(theta1, theta2_u, theta3_u).T @ R @ M.T
    theta4_u, theta5_u, theta6_u = negXYX_EulerAngles(wrist_rotation)
    
    # Elbow down solution
    theta2_d = gamma - alpha
    theta3_d = -phi1 + phi2
    
    # Wrist rotation
    wrist_rotation = (M @ R.T @ arm_fk(theta1, theta2_d, theta3_d)).T
    theta4_d, theta5_d, theta6_d = negXYX_EulerAngles(wrist_rotation)
    

    # theta2 and theta3 are flipped because of a minor error in positive/negative rotation direction
    return (
        [theta1, -theta2_u, -theta3_u, theta4_u, theta5_u, theta6_u],
        [theta1, -theta2_d, -theta3_d, theta4_d, theta5_d, theta6_d]
    )
    
# Testing
test_target_frame = np.array([
    [0, 0, 1, 300e-3],
    [0, -1, 0, 400e-3],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
])
ik_analytic(test_target_frame)

([0.8110335719191258,
  -0.03355247677107365,
  1.5158581132113478,
  -2.3286016483044136,
  -1.6317023788131908,
  -3.0773574620378215],
 [0.8110335719191258,
  1.3325454203501432,
  -1.3495756494344653,
  -1.5546198023739004,
  -2.330421330379375,
  -1.5473083863605865])

## Numerical IK 

In [7]:
def adjusted_ik_solution(joint_angles):
    return [
        joint_angles[0], # theta1
        joint_angles[1], # theta2
        joint_angles[2], # theta3
        -joint_angles[3], # theta4
        -joint_angles[4], # theta5
        -joint_angles[5] # theta6
    ]



# The joint screw axes in the end-effector frame when the manipulator is at the home position
Blist = np.array([
    [ -1,  0, 0,      0    , -L1-L2-L3-L4,  0 ],    # B_1
    [  0,  1, 0,  -L2-L3-L4,       0     , -H3],    # B_2
    [  0,  1, 0,   -L3-L4  ,       0     , -H3],    # B_3
    [  0,  0, 1,      0    ,       0     ,  0 ],    # B_4
    [  0,  1, 0,     -L4   ,       0     ,  0 ],    # B_5
    [  0,  0, 1,      0    ,       0     ,  0 ]]).T # B_6


M = np.array([[0,0,-1,L1+L2+L3+L4], [0,-1,0,0], [-1,0,0, H1+H3], [0,0,0,1]])

target_frame = np.array([
    [0, 0, 1, 300e-3],
    [0, -1, 0, 400e-3],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
])


thetalist0 = np.array(adjusted_ik_solution(ik_analytic(target_frame)[0]))

eomg = 0.00001
ev = 0.00001



nSolution = np.degrees(mr.IKinBody(Blist, M, target_frame, thetalist0, eomg, ev)[0])
nSolution = np.concatenate((nSolution[0:3],nSolution[3:]*-1))
nSolution


array([  46.46882575,   -6.97942269,  -67.99305218, -132.53668442,
       -100.28714469, -168.98585972])

### Comparing Numerical and Analytic solution


In [8]:
target_frame = np.array([
    [0, 0, 1, 300e-3],
    [0, -1, 0, 400e-3],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
])

thetalist0 = np.array(adjusted_ik_solution(ik_analytic(target_frame)[0]))

aSolution = ik_analytic(target_frame)[0]
nSolution = mr.IKinBody(Blist, M, target_frame, thetalist0, eomg, ev)[0]
nSolution = np.concatenate((nSolution[0:3],nSolution[3:]*-1)) 
#np.allclose(aSolution, nSolution, rtol=2e-01, atol=2e-01, equal_nan=False)



aSolution, nSolution

# Works sometimes with high enough tolerance 
# => either some kinematical mistake, numerical solver doesnt work as intended, or both

([0.8110335719191258,
  -0.03355247677107365,
  1.5158581132113478,
  -2.3286016483044136,
  -1.6317023788131908,
  -3.0773574620378215],
 array([ 0.81103401, -0.12181391, -1.18670263, -2.31320152, -1.75034087,
        -2.94935964]))

In [9]:
target_frame = np.array([
    [1, 0, 0, 400e-3],
    [0, 1, 0, 200e-3],
    [0, 0, 1, -400e-3],
    [0, 0, 0, 1],
])

thetalist0 = np.array(adjusted_ik_solution(ik_analytic(target_frame)[0]))

aSolution = ik_analytic(target_frame)[0]
nSolution = mr.IKinBody(Blist, M, target_frame, thetalist0, eomg, ev)[0]
nSolution = np.concatenate((nSolution[0:3],nSolution[3:]*-1)) 
np.allclose(aSolution, nSolution, rtol=10e-05, atol=1e-03, equal_nan=False)

aSolution , nSolution


([0.4636476090008061,
  -0.8042744630056224,
  2.2024768033094917,
  -0.0,
  -2.968998667098765,
  2.677945044588987],
 array([19.31320592, -0.80724187, -0.48413007,  9.42485933, 18.57013154,
         6.74673343]))

In [10]:
target_frame = np.array([
    [0, 0, 1, 300e-3],
    [0, -1, 0, 400e-3],
    [1, 0, 0, -400e-3],
    [0, 0, 0, 1],
])

thetalist0 = np.array(adjusted_ik_solution(ik_analytic(target_frame)[0]))

aSolution = ik_analytic(target_frame)[0]
nSolution = mr.IKinBody(Blist, M, target_frame, thetalist0, eomg, ev)[0]
nSolution = np.concatenate((nSolution[0:3],nSolution[3:]*-1)) 

np.allclose(aSolution, nSolution, rtol=10e-01, atol=1e-03, equal_nan=False)

aSolution,  nSolution



([0.8110335719191258,
  -0.876096080563673,
  1.9372544611632807,
  -2.2631348169151257,
  -1.9134781197386457,
  -2.7566199850876805],
 array([  23.81484404,  -12.50924871,  -13.76448573,  507.2538073 ,
          24.30673393, -518.8457817 ]))

## Visualizing

In [11]:
scene2 = robot_scene.create_scene()

In [12]:


def make_target_frame(x, y, z, yaw, pitch, roll):
    c_yaw = np.cos(yaw)
    s_yaw = np.sin(yaw)
    R_yaw = np.array([[1, 0, 0], [0, c_yaw, -s_yaw], [0, s_yaw, c_yaw]])
    
    c_pitch = np.cos(pitch)
    s_pitch = np.sin(pitch)
    R_pitch = np.array([[c_pitch, 0, s_pitch], [0, 1, 0], [-s_pitch, 0, c_pitch]])
    
    c_roll = np.cos(roll)
    s_roll = np.sin(roll)
    R_roll = np.array([[c_roll, -s_roll, 0], [s_roll, c_roll, 0], [0, 0, 1]])
    
    target_frame = np.eye(4)
    target_frame[0:3, 0:3] = R_yaw @ R_pitch @ R_roll
    target_frame[0:3, 3] = [x, y, z]
    return target_frame

def show_target(scene, target_frame):
    scene.target.matrix = target_frame.flatten(order="F").tolist()

# Interactive loop
def interact_ik(x, y, z, yaw, pitch, roll, elbow_position, must_show_robot, must_show_frames, must_show_axes):
    # Convert the interactive sliders value into a transformation matrix
    target_frame = make_target_frame(x*1e-3, y*1e-3, z*1e-3, np.radians(yaw), np.radians(pitch), np.radians(roll))
    
    # Solve the inverse kinematics
    if elbow_position == "elbow_up":
        (joint_angles, _) = ik_analytic(target_frame)
    elif elbow_position == "elbow_down":
        (_, joint_angles) = ik_analytic(target_frame)
    
    # Do the forwards kinematics to check the solution and to display the robot
    (frames, transforms) = fk_poe(screw_axes, joint_angles, frames_at_rest)
    
    print("Target:\n", target_frame)
    print("End effector:\n", frames[-1])
    print("Target reached:", np.allclose(frames[-1], target_frame))
    print("Solution:\n", joint_angles)
    show_target(scene2, target_frame)
    show_robot(scene2, transforms, must_show_robot)
    show_frames(scene2, frames, must_show_frames)
    show_axes(scene2, frames, must_show_axes)

# Start the interaction
interactive2 = ipywidgets.interactive(
    interact_ik,
    x=ipywidgets.IntSlider(description="X (mm)", min=-1000, max=1000, step=1, value=300),
    y=ipywidgets.IntSlider(description="Y (mm)", min=-1000, max=1000, step=1, value=400),
    z=ipywidgets.IntSlider(description="Z (mm)", min=-1000, max=0, step=1, value=0),
    yaw=ipywidgets.IntSlider(description="Yaw (°)", min=-180, max=180, step=1, value=180),
    pitch=ipywidgets.IntSlider(description="Pitch (°)", min=-90, max=90, step=1, value=-90),
    roll=ipywidgets.IntSlider(description="Roll (°)", min=-180, max=180, step=1, value=0),
    elbow_position=ipywidgets.ToggleButtons(options=["elbow_up", "elbow_down"]),
    must_show_robot=ipywidgets.Checkbox(description="Show Robot", value=True),
    must_show_frames=ipywidgets.Checkbox(description="Show Frames", value=False),
    must_show_axes=ipywidgets.Checkbox(description="Show Axes", value=False),
)

display(ipywidgets.HBox([
    scene2.viewer.renderer,
    ipywidgets.VBox(interactive2.children)
]))